## 랜덤포레스트
- 학습 데이터로 여러 의사결정 트리를 구성해 분석

In [1]:
import pandas as pd
data=pd.read_csv('C:\\Users\\user\\Desktop\\VS code\\빅데이터 분석기사\\data\\breast-cancer-wisconsin.csv',encoding='utf-8')
data.head()

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [2]:
#독립변수와 종속변수 나누기
X=data[data.columns[1:10]]
y=data[['Class']]

In [3]:
# train, test 데이터셋을 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,stratify=y,random_state=42)

In [4]:
#MinMaxScaler: 최소 0, 최대 1 정규화
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X_scaled_train=scaler.fit_transform(X_train)
X_scaled_test=scaler.fit_transform(X_test)

In [5]:
#랜덤 포레스트 모델 적용
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(X_scaled_train,y_train)
pred_train=model.predict(X_scaled_train)
model.score(X_scaled_train,y_train)

C:\Users\user\AppData\Local\Temp\ipykernel_108672\485299164.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_scaled_train,y_train)


1.0

In [6]:
#오차행렬로 확인
from sklearn.metrics import confusion_matrix
confusion_train=confusion_matrix(y_train,pred_train)
confusion_train
#정상 중 2명 오분류, 환자중 6명 오분류

array([[333,   0],
       [  0, 179]], dtype=int64)

In [7]:
from sklearn.metrics import classification_report
cfreport_train=classification_report(y_train,pred_train)
cfreport_train

'              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00       333\n           1       1.00      1.00      1.00       179\n\n    accuracy                           1.00       512\n   macro avg       1.00      1.00      1.00       512\nweighted avg       1.00      1.00      1.00       512\n'

In [8]:
pred_test=model.predict(X_scaled_test)
model.score(X_scaled_test,y_test)

0.9707602339181286

In [11]:
#grid search: 하이퍼 파라미터 값 조정
param_grid={'n_estimators':range(100,500,100),
            'max_features':['auto','sqrt','log2']}
from sklearn.model_selection import GridSearchCV
grid_search=GridSearchCV(RandomForestClassifier(),param_grid,cv=5)
grid_search.fit(X_scaled_train,y_train)

c:\Users\user\anaconda3\envs\baek\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\user\anaconda3\envs\baek\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\user\anaconda3\envs\baek\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\user\anaconda3\envs\baek\lib\si

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': range(100, 500, 100)})

In [12]:
print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.score(X_scaled_test,y_test))

{'max_features': 'log2', 'n_estimators': 400}
0.976527698458024
0.9649122807017544


In [13]:
from scipy.stats import randint
param_distribs={'n_estimators':randint(low=100,high=1000),'max_features':['auto','sqrt','log2']}
from sklearn.model_selection import RandomizedSearchCV
random_search=RandomizedSearchCV(RandomForestClassifier(),param_distributions=param_distribs,n_iter=20,cv=5)
random_search.fit(X_scaled_train,y_train)

c:\Users\user\anaconda3\envs\baek\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\user\anaconda3\envs\baek\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\user\anaconda3\envs\baek\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\user\anaconda3\envs\baek\lib\site-packages\sklearn\model_selection\_validation.py:686: DataCon

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=20,
                   param_distributions={'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001F67C680490>})

In [14]:
print(random_search.best_params_)
print(random_search.best_score_)
print(random_search.score(X_scaled_test,y_test))

{'max_features': 'sqrt', 'n_estimators': 241}
0.9745859508852085
0.9649122807017544


In [17]:
# 회귀
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor()
model.fit(X_scaled_train,y_train)
pred_train=model.predict(X_scaled_train)
model.score(X_scaled_train,y_train)

C:\Users\user\AppData\Local\Temp\ipykernel_108672\3246827105.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_scaled_train,y_train)


0.9830948177227506

In [18]:
pred_test=model.predict(X_scaled_test)
model.score(X_scaled_test,y_test)

0.8514354054054054

In [20]:
import numpy as np
from sklearn.metrics import mean_squared_error
MSE_train=mean_squared_error(y_train,pred_train)
MSE_test=mean_squared_error(y_test,pred_test)
np.sqrt(MSE_train)
np.sqrt(MSE_test)

0.18394952269735518